In [ ]:
#Импортируем нужные библиотеки

import pandas as pd
import numpy as np
from scipy.signal import argrelextrema

import seaborn as sns 
import matplotlib.pyplot as plt

import datetime
import warnings


In [ ]:
#Ставим кол-во выводимых строк и столбцов 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#Отключаем предупреждения

warnings.filterwarnings('ignore')


In [ ]:
#Кастомизируем стиль и размер наших будующих графиков

plt.rcParams['figure.figsize']=[18,13]
sns.set_style("whitegrid")

%matplotlib inline

In [ ]:
#Загружаем данные

with open('plr_smpl_attempts.csv') as f:
    df = pd.read_csv(f,sep=';')

In [ ]:
df.head(2)

In [ ]:
#Создаем копию

dx = df.copy()

In [ ]:
dx.head(2)

In [ ]:
df.dtypes

In [ ]:
#Пустые значения 

dx.isnull().sum()

In [ ]:
#Удаляем дубликаты

dx.drop_duplicates(inplace = True)

#Проверяем что дубликатов нет

dx[dx.duplicated()]

In [ ]:
#Обзорная таблица

observe = pd.DataFrame([{
                        'Total uids/attempts':dx['uid'].count(),
                        'Unique uids':dx['uid'].nunique(),
                        'Unique rounds':dx['level'].nunique(),
                        'False attempts':dx['uid'][(dx['action']=='failed')].count(),
                        'Positive attempts':dx['uid'][(dx['action']=='completed')].count()
                        }])
observe['Attempts per user'] = round(observe['Total uids/attempts']/observe['Unique uids'],0)
observe

In [ ]:
#Преобразовываем event_time и добавляем новые переменные

dx['normal_time'] = pd.to_datetime(dx['event_time'], utc=True, unit='ms')
dx['dt'] = dx['normal_time'].dt.date
dx['time'] = dx['normal_time'].dt.time


In [ ]:
dx.head(2)

In [ ]:
#Сколько уникальных пользователей в день 

grouped = dx[['dt','uid']].groupby(['dt'],as_index=False).nunique()

#Процентная разница в уникальных пользователях к предыдущему дню по дням

grouped['Percent'] = round((grouped['uid']-grouped['uid'].shift(1))/grouped['uid']*100,2)

grouped.head(5)

In [ ]:
#Выделим старых и новых/вернувшихся пользователей при условии что: 
#Новые/вернувшиеся - те кто впервые играет в рассматриваемый день а значит его dt=min(dt),под это условие подходят и вернувшиеся юзеры

grouped_1 = dx[['uid','dt']].groupby(['uid'],as_index=False).min().rename(columns = {'dt': 'min_dt'})
grouped_2 = grouped_1.merge(dx[['uid','dt']],how='left',left_on = 'uid',right_on='uid')
grouped_2.drop_duplicates(inplace = True)
grouped_2['User_Status'] = ['New' if x==y else 'Old' for x,y in zip(grouped_2['min_dt'],grouped_2['dt'])]
grouped_3 = grouped_2[['dt','User_Status','uid']].groupby(['dt','User_Status'],as_index=False).nunique()

grouped_3.head(5)

In [ ]:
#Распределение уникальных пользователей по дням

plt.rcParams['figure.figsize']=[18,13]
plt.subplot(4, 1, 1)

sns.barplot(data = grouped,x='dt',y='uid')

plt.title('Unique users per day')

#Процент изменения пользователей по дням

plt.subplot(4, 1, 2)

sns.lineplot(data = grouped,x='dt',y='Percent')

#Добавим подписи значений на график 

for i,j in zip(grouped['dt'],grouped['Percent']):
    plt.annotate(str(j),(i,j),textcoords = 'offset points',xytext=(0,10),ha='center')


plt.title('Percent of unique users past day')

#Новые/вернувшиеся и старые пользователи

plt.subplot(4, 1, 3)

sns.barplot(data = grouped_3,x='dt',y='uid',hue='User_Status')

plt.title('New/Old users per day')

#Отдельно выделим новых/вернувшихся пользователей

plt.subplot(4, 1, 4)

sns.barplot(data = grouped_3[(grouped_3['User_Status']=='New')],x='dt',y='uid')

plt.title('New users per day')

plt.show()

###### Выводы из полученных данных:
<ol>
<li>Пользователи +- стабильно играют в игру по дням хотя и намечается спад;</li>
<li>Новых пользователей ,которые впервые сыграли одну игру в разы меньше чем старых пользователей;</li>
<li>Кол-во новых пользователей по дням +- одинаковый изо дня в день ,хоть и есть некоторые всплески;</li>
<li>32% пользователей играет только один день!;</li>
</ol>

In [ ]:
#Пользователи которые играют только один день
#Уберем из выбокри пользователей где их min dt = максимальному дню нашего датасета 
#Так как если он впервые сыграл в последний день нашего датасета у него не было возможности сыграть больше

Test = dx[['uid','dt']].groupby(['uid'],as_index=False).agg({'dt':['max','min','nunique']})
Test['Delta_Days'] = (Test['dt']['max']-Test['dt']['min']).dt.days
Test = Test[(Test['dt']['min']<dx['dt'].max())]
Test.head(5)

In [ ]:
n, bins, patches = plt.hist(data = Test,x=list(Test['dt']['nunique']),bins=31)
plt.show()

In [ ]:
#Соберем таблицу из данных на гистограмме

Days_count = list(Test['dt']['nunique'])
uid_count = list(Test['uid'])
uid_days = pd.DataFrame(list(zip(Days_count,uid_count)),columns = ['Days_count','uid'])
uid_days_1 = uid_days[['Days_count','uid']].groupby(['Days_count'],as_index=False).count().sort_values(by='uid',ascending=False)
uid_days_1['Delta%'] = round((uid_days_1['uid']-uid_days_1['uid'].shift(1))/uid_days_1['uid']*100,1)
uid_days_1['Uid per total uids'] = round(uid_days_1['uid']/Test['uid'].nunique()*100,1)
uid_days_1.head(5)

<ol>
<li>61261 или 31.2% от общего числа пользователей играют только один день;</li>
<li>2 дня играют уже 28377 пользователей, что на 116% меньше тех кто играет 1 день;</li>
</ol>

In [ ]:
#Расчеты относительно пользователя и его уровней

In [ ]:
Test_1 = dx[['uid','level']].groupby(['uid'],as_index=False).agg({'level':['count','nunique','min','max']})
Test_1['Played Rounds'] = Test_1['level']['max']-Test_1['level']['min']+1
Test_1['Attempts per game'] = round(Test_1['level']['count']/Test_1['Played Rounds'],0)

#Создана на основании того, что на последующих шагах мы увидим расхождения 
Test_1['Attempts per game 2'] = round(Test_1['level']['count']/Test_1['level']['nunique'],0)

In [ ]:
Test_1.head(5)

<ol>
<li>nunique должен равняться Played Rounds но в некоторых случаях это не так.</li>
<li>Получаем ,что у нас есть пропущенные раунды у пользователей</li>
</ol>

In [ ]:
#Кейсы где пропущены номера раундов 

Test_1[(Test_1['level']['nunique'] != Test_1['Played Rounds'])]

In [ ]:
#Подсчет по раунду

In [ ]:
#Создадим таблицу для будущего подсчета уникальных пользователей в каждом раунде

grouped_1 = dx[['level','uid']].groupby(['level'],as_index=False).nunique()
#grouped_1['cumsum_uid'] = grouped_1['uid'].cumsum()
#grouped_1['Trash_Hold'] = round(grouped_1['cumsum_uid']/dx['uid'].nunique()*100,0)
#grouped_1['Trash_Hold_level'] = [x if y>99.0 else 0 for x,y in zip(grouped_1['level'],grouped_1['Trash_Hold'])]
#min_level = grouped_1[['level']][(grouped_1['Trash_Hold_level']!=0)].min()

grouped_1['local_min']=grouped_1['uid'][(grouped_1['uid'].shift(1) > grouped_1['uid']) &\
                                    (grouped_1['uid'].shift(-1) > grouped_1['uid'])]

grouped_1['local_max']=grouped_1['uid'][(grouped_1['uid'].shift(1) < grouped_1['uid']) &\
                                    (grouped_1['uid'].shift(-1) < grouped_1['uid'])]

In [ ]:
grouped_1.head(5)

In [ ]:
grouped_1[(grouped_1['local_min']>0)]

In [ ]:
grouped_1[(grouped_1['local_max']>0)]

In [ ]:
#Кол-во уникальных пользователей в каждом раунде

sns.scatterplot(data=grouped_1,x='level',y='uid')

plt.axvline(x=50,color = 'g',label = 'Level 50')
plt.axvline(x=100,color = 'r',label = 'Level 100')
plt.axvline(x=169,color = 'g',label = 'Level 169')

#Подкрасим наши локальные минимумы и максимумф

plt.scatter(grouped_1.index, grouped_1['local_min'], c='g',label = 'Local min')
plt.scatter(grouped_1.index, grouped_1['local_max'], c='r',label = 'Local max')

plt.legend()
plt.show()

###### Из графика видно ,что:
<ol>
<li>Есть резкое падение после 1го уровня до 50-го;</li>
<li>Большой/аномальный retention пользователей с 50-го по 100-й уровень,по большей части обусловлен именно retention пользователей в конкретном раунде ,а не переходом из других.Это можно будет понять из расчетов ниже;</li>
<li>После 100 уровня кол-во пользователей резко снижается;</li>
<li>Основная масса пользователей находится в диапозоне от 1 до 169 уровня;</li>
</ol>

In [ ]:
#Churn rate

In [ ]:
#Выделим пользователей по стартовому уровню и посмотрим как они продвигаются по игре 

Temp_1 = dx[['uid','level']].groupby(['uid'],as_index=False).min().rename(columns={'level':'start_level'})
Temp_2 = dx[['uid','level']].merge(Temp_1,how='left',left_on='uid',right_on='uid')
Temp_2.drop_duplicates(inplace = True)
Temp_3 = Temp_2[['start_level','level','uid']].groupby(['start_level','level'],as_index=False).nunique()


### Воронка перехода 

In [ ]:

#Рассмотрим новых пользователей ,которые начали игру с 1-го уровня и их 20 последующих уровней

Temp_4 = Temp_3[(Temp_3['start_level']==1)].reset_index()
Temp_4['Delta'] = [round((Temp_4['uid'][n]-Temp_4['uid'][0])/Temp_4['uid'][0]*100,2) for n in range(len(Temp_4['uid']))]

sns.lineplot(data = Temp_4,x='level',y='Delta')
plt.show()
Temp_4.head(20)

In [ ]:
#20% новых пользователей уходят после 4-го уровня 50% уходят после 14-го уровня

In [ ]:
#Можно проверить любой уровень и разделить пользователей на перешедших в уровень и на неожиданно вернувшихся

Temp_3[(Temp_3['level']==94)]


In [ ]:
#Можно улучшить график, который уже был выше по уникальным пользователям

Temp_3['status'] = [ 'new' if x==y else 'transfered' for x,y in zip(list(Temp_3['level']),list(Temp_3['start_level']))]
Temp_5 = Temp_3[['level','status','uid']].groupby(['level','status'],as_index=False).sum()


In [ ]:

sns.scatterplot(data=Temp_5[(Temp_5['status']=='new')],x='level',y='uid',color='g',label='New')
sns.scatterplot(data=Temp_5[(Temp_5['status']=='transfered')],x='level',y='uid',color='r',label='Transfered')

plt.axvline(x=50,color = 'g',label = 'Level 50')
plt.axvline(x=100,color = 'r',label = 'Level 100')
plt.axvline(x=169,color = 'g',label = 'Level 169')

plt.legend()
plt.show()

In [ ]:
#Посчитаем процент потерь при переходе с одного уровня на следующий по каждому уровню

Cum_DF = pd.DataFrame()

for i in list(set(Temp_3['start_level'])):
    Temp_5 = Temp_3[(Temp_3['start_level']==i)].reset_index()
    Temp_5['Delta'] = [round((Temp_5['uid'][n]-Temp_5['uid'][0])/Temp_5['uid'][0]*100,2) for n in range(len(Temp_5['uid']))]
    Cum_DF=pd.concat([Temp_5,Cum_DF])

In [ ]:
#Возьмем первые 20 уровней с которых начинает пользователь и посмотрим % потерь при переходе на следующий уровень

K = Cum_DF[(Cum_DF['level']==Cum_DF['start_level']+1)].reset_index()
T = K.sort_values(by='level').head(20)

In [ ]:

sns.lineplot(data = T,x='level',y='Delta')
for i,j in zip(T['level'],T['Delta']):
    plt.annotate(str(j),(i,j),textcoords = 'offset points',xytext=(0,10),ha='center')
plt.show()

In [ ]:
#Самый плохой процент перехода с 7 на 8 уровень
#Как мы поймем дальше 7 уровень - еще один сложный уровень в игре

### #Метрика сложности уровня - среднее число попыток на уровень

In [ ]:

#Сколько попыток делает пользователь на каждом уровне

Attempts = dx[['level','uid']].groupby(['level'],as_index=False).agg({'uid':['count','nunique']})
Attempts['mean_attepts_round'] = round(Attempts['uid']['count']/Attempts['uid']['nunique'],0)

In [ ]:
#Попытки по уровням

plt.subplot(2, 1, 1)

sns.scatterplot(data=Attempts,x=Attempts['level'],y=Attempts['uid']['count'])

#Среднее кол-во попыток на определенном уровне

plt.subplot(2, 1, 2)

sns.lineplot(data=Attempts,x=Attempts['level'],y=Attempts['mean_attepts_round'])

plt.legend()
plt.show()

In [ ]:
set(dx['action'])

### Метрика сложности уровня - Сложность уровня число плохих юзеров(завалили попытку) в раунде к числу хороших юзеров(прошли раунд).

In [ ]:

Failed = dx[['level','uid']][(dx['action']=='failed')].groupby(['level'],as_index = False).agg({'uid':['count','nunique']}).\
rename(columns = {'count':'Failed_Attempts','nunique':'Users_Failed'})

Completed = dx[['level','uid']][(dx['action']=='completed')].groupby(['level'],as_index = False).agg({'uid':['count','nunique']}).\
rename(columns = {'count':'completed_Attempts','nunique':'Users_completed'})

joined = Failed.merge(Completed,how='inner')
joined['Failled%'] = round(joined['uid']['Users_Failed']/joined['uid']['Users_completed']*100,2)

#1 пользователь может совершить несколько хороших попыток?Если пройти уровень несколько раз

sns.lineplot(data = joined,x='level',y='Failled%')
plt.show()


In [ ]:
joined

In [ ]:
#Сколько уровней сыграл уникальный пользователь

n, bins, patches = plt.hist(data = Test_1,x=list(Test_1['level']['nunique']),bins=500)
plt.axhline(y = n.max(), color = 'r', linestyle = '--',label = f'{n.max()}: Число уникальных пользователей')
plt.axvline(x=np.argmax(n)+1,color = 'g',label = f'{np.argmax(n)+1}: Кол-во сыгранных раундов')
plt.legend()
plt.show()

#Число bins=500 - кол-во раундов, значит 1 деление гистограммы соответствует 1 раунду

<ol>
<li>Большенство пользователей играют один уровень после чего не переходят на другой.</li>
</ol>

In [ ]:
#Расчет % уникальных пользователей относительно предыдущего уровеня

Unique_Users = pd.DataFrame(list(zip([round(i,0) for i in bins],n)),columns = ['rounds','unique_users'])
Unique_Users['Delta%'] = round((Unique_Users['unique_users']-Unique_Users['unique_users'].shift(1))/Unique_Users['unique_users']*100,1)
Unique_Users['Unique user per played round/Total%'] = round(Unique_Users['unique_users']/dx['uid'].nunique()*100,1)

In [ ]:
Unique_Users.head(5)

In [ ]:
#Кол-во уникальных пользователей по кол-во сыгранных раундов ко всем уникальным пользователям

sns.lineplot(data = Unique_Users,x='rounds',y='Unique user per played round/Total%')

plt.show()

<ol>
<li>Резкое падение уникальных пользователей (-126%) после одного сыгранного уровня</li>
</ol>

In [ ]:
#Сколько играет пользователь

n, bins, patches = plt.hist(data=Test_1,x=list(Test_1['level']['count']),bins=200)
plt.axvline(x=np.argmax(n)+1,color = 'g',label = f'{np.argmax(n)+1}: Кол-во сыгранных игр')
plt.legend()
plt.show()

#Тут число Bins=200 ,оно не соответствует общему числу попыток 

In [ ]:
#Та же гистограмма только в табличном виде и без Bins

level_count = list(Test_1['level']['count'])
uid = list(Test_1['uid'])
A = pd.DataFrame(list(zip(level_count,uid)),columns = ['level_count','uid'])
AB = A[['level_count','uid']].groupby(['level_count'],as_index=False).count().sort_values(by='uid',ascending=False)
AB['Delta%'] = round((AB['uid']-AB['uid'].shift(1))/AB['uid']*100,1)
AB['Uid per total uids'] = round(AB['uid']/dx['uid'].nunique()*100,1)
AB

<ol>
<li>Большенство пользователей 14534=7.4% от всех пользователей играет 1 игру</li>
<li>2 игры играют уже  9135=4.7% от всех пользователей или на 59% меньше чем играют первую игру</li>
</ol>

In [ ]:
#Сколько совершает попыток уникальный юзер на один раунд

n, bins, patches = plt.hist(data=Test_1,x='Attempts per game 2',bins=200)
plt.axvline(x=np.argmax(n)+1,color = 'g',label = f'{np.argmax(n)+1}: Кол-во попыток на уровень')
plt.legend()

plt.show()

#Тут число Bins=200 ,оно не соответствует общему числу попыток и с помощью np.argmax(n) мы не можем определить точное число попыток

In [ ]:
#Проделаем тоже самое,что и в предыдущих шагах
Test_1[['Attempts per game 2','uid']].groupby(['Attempts per game 2'],as_index=False).count().\
                                                                                    sort_values(by='uid',ascending=False).head(5)


<ol>
<li>Среднее кол-во попыток на уровень = 2</li>
</ol>

## Выводы

<ol>

<li>61261 или 31% пользователей играют лишь 1 день после чего не возвращаются,2 дня играют уже 28377 пользователей, что на 116% меньше тех кто играет 1 день.</li>
<li>После одного сыгранного уровня, кол-во пользователей сокращается более чем в 2 раза ,на -126.4% относительно игроков которые сыграли 2 уровня</li>
<li>Менее 1% пользователей играет каждый день</li>
<li>#20% новых пользователей(кто начал игру с 1-го уровня) уходят после 4-го уровня, 50% уходят после 14-го уровня</li>
<li>Уход после 4-го уровня можно обусловить тем,что 5-й уровень первый сложный уровень в игре</li>
<li>Большенство пользователей не переходят на следующий уровень внезависимости от того с какого уровня они начали,график того ,что пользователи в основном играют 1 раунд это подтверждает</li>
<li>14534 или 7.4% от всех пользователей играет 1 игру и больше не играют</li>
<li>В зависимости от уровня пользователь чаще всего совершает от 1 до 3 попыток</li>
<li>После 100 уровня кол-во пользователей резко снижается</li>

<li>Так же были замечены странные пользователи, которые перескочили уровни, хотя вроде должно все идти подряд</li>
</ol>


## Пожелания

<ol>
<li>Увеличивать приток новых пользователей.</li>
<li>Работать над удержанием новых пользователей особенно до 14-го уровня</li>
<li>Следить за переходом на следующий уровень</li>

</ol>